# Outcome Predictions & Limited Dependent Variable Models for the NFL

Sean Tibay

ECON 570: Big Data Econometrics

Professor Ida Brigitta Johnsson

## Introduction

The purpose of this research is to build a forecasting model in order to predict sports outcomes, specifically outcomes of games in the National Football League (NFL). Unlike a causal model, this project seeks to examine predicting power as opposed to estimated coefficients, and as such, the focus of this assignment will be emphasized on the analysis of the prediction accuracy of several different forecasting models.

To provide some context to the basic work that is outlined in this project, regression models will take the form of a limited, binary dependent variable, where the predicted values from the model will indicate the probability value of a certain event happening. These types of problems are interesting to research because of their application into other micro-economic and consumer choice theory – attempting to model the probability of certain events happening based on a set of observable explanatory variables; but in the case of predicting sports outcomes, this research attempts to delve into the ever-growing market of sports entertainment both in the field of sports analytics, and gambling & odds-making. By attempting to model game outcomes using existing data, we can compare prediction results to published odds and sports lines to examine the specific processes of sports book-making by professional betting organizations, as well as determine whether outperforming such bookmakers is a possibility.

In terms of the specific objective of this project, I aim to answer the following research questions: 

1). What set of explanatory variables create the best forecasting model in terms of RMSE as well as out-of-sample prediction? 

2). Is it possible to improve on the forecasting accuracy of a standard OLS model with the use of Probit or Logit models by constraining predicted values within the range of possible outcomes 0 to 1? 

3). Does the best model from our identified list of methods have the capability or potential to outperform professional sportsbooks or published odds?

With the rapid legalization of sports-betting in numerous states across the US and globally, this area of research provides excellent potential and opportunity for economic activity both from the perspective of professional organizations in optimizing forecasting models, but also from the perspective of consumers in seeking potentially statistically supported arbitrage opportunities.


## Literature Review

Dr. Sascha Wilkens examines several models in predicting the outcome of professional tennis matches, using data from 2010 - 2019, with the main predicting model being the Logistic Regression. By incorporating match statistics and betting market data into the set of explanatory variables, Wilkens finds that beyond the betting market data, historical match statistics and player demographics (such as age and experience) adds little to insignificant effects on prediction accuracy. In fact, in attempting to develop a profit strategy in identifying discrepancies in predicted values and published odds, Wilkens finds a negative long run return on investment (Wilkens).

Sindre Hansen utilizes an extended Bradley-Terry model to forecast game results in the Premier League for Soccer by incorporating a list of features including historical statistics as well as player salaries. In this research, Hansen finds that several identified models produced results generally in-line with published odds from sportsbooks, and utilizing a value threshold (ex. 20% discrepancy between predicted probability and published odds), Hansen finds that all but one model produces positive returns on bets made for the testing period from 2013 - 2014. 

For this project, we combine certain methodologies from both papers in answering our research questions – specifically in regards to the type of regressors utilized, limited dependent variable regression models, and analysis on possible edge over betting markets.


## Data

The data source of this project is compiled from the ESPN database, utilizing a python script to web-scrape over 1,800 individual, unique events from the 2014 season up to the 2020 season – each year containing 256 regular season games. From each game, several statistics are extracted for our dataset: outcome, QBR, Rushing Avg., Turnovers, Points Scored, Sacks, 3rd Down Conversions (game statistics are collected for both teams). 

Then, the data is transformed such that the model is capable of ex-ante prediction, as opposed to ex-poste prediction. Since creating forecasts for our regressors are not feasible due to the high variability of game statistics game to game as well as the opponent dependency of certain statistics, this project resorts to modeling game outcomes on a series of distributed lags of regressors such that real-time prediction is possible with historical data.

To create a simplified version of distributed lag models for the regressors, we transform each variable into two formats: Year-to-Date averages, and Last-3-Week averages. Both of these measures are equal weight distributions of all previous observations or last three observations respectively in each year. (Note: The NFL season is 16 games for each team). The purpose of this distinction is to test whether results are driven moreso by a long-run performance of a team, or whether a recent average performance is a better indicator for present outcomes.

But as a result of doing so, sample restrictions must also be made, seeing as the first three observations of each year are removed as a result of using a 3-week average. To generate enough differentiability between the Year-to-date average model and the 3-week average, we will not include the first 5 observations of the observed variables in each year such that our model only includes games 6-16 for each team. Furthermore, to analyze the out-of-sample prediction power of the model, 2014 - 2018 is included as the training data, and 2019 - 2020 is excluded, and to be used for out-of-sample predictions analysis. Finally, we make one final restriction that removes games that end in a tie such that the model is able to estimate the binary limited dependent variables, and observed outcomes are limited to 0 and 1.

Actual published odds and sportsbook betting market data are collected from a third party site (www.sportsbookreviewsonline.com) that aggregates odds and probabilities from multiple betting organizations and publishes datasets for every game from 2008 – 2020. The same transformations and observation restrictions are performed on this dataset.

Relevant summary statistics are displayed below.

In [31]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.tools.eval_measures import rmse
from math import e

# Importing Main Data Set
df = pd.read_excel('570Dataset.xlsx')

# Data Cleaned such that td stands for Training Data from 2014-2019
# Also removes Pre-week 5, and games that end in Tie
td = df[(df['Season'] < 2019) & (df['Week']>5) & (df['hwlBIN'] != 0.5)]

#### Data Glossary & Definitions:

First letter: (h,a) indicating statistic for home or away team. (Only home team stats are displayed since there are no discernable differences between home stats vs away stats in a large sample.

Second letter: (y,l3) indicating year-to-date stats or last-3-weeks stats.

Win_PCT = Winning Percentage

QBR = Quarterback Rating

RYA = Rushing Yards Avg.

TO = Turnovers

PS = Points Scored

3dc = 3rd Down Conversions

SA = Sacks Allowed

PA = Points Allowed

S = Sacks

3DCA = 3rd Down Conversions Allowed

QBRA = QBR Allowed

RYAA = RYA Allowed

TOA = Turnovers Allowed

Note: Allowed Stats are not the inverse of said stats. They are stats on the opposite side of play (Offense vs. Defense).


#### Year-to-Date Variables:
Equal Weight Averages of Historical performance up to present day in each observation.

In [43]:
# Extensive Summary Statistics

Year_to_Date_Variables = td.iloc[: , 6:19]
display(Year_to_Date_Variables.describe())

,hyWin_PCT,hyqbr,hyrya,hyto,hyps,hy3dc,hysa,hypa,hys,hy3dca,hyqbra,hyryaa,hytoa
count,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000
mean,0.496048,55.019001,4.118114,1.416374,22.895154,5.193081,2.319136,23.045312,2.313102,5.226617,55.127748,4.121004,1.414491
std,0.212066,11.149302,0.501883,0.459857,4.581520,0.834046,0.731464,3.787831,0.624753,0.792414,6.612980,0.488301,0.459158
min,0.000000,25.685714,2.520000,0.000000,10.875000,2.333333,0.500000,12.833333,0.200000,2.571429,29.580000,2.075000,0.333333
25%,0.357143,47.268750,3.780000,1.125000,19.818182,4.611538,1.724026,20.181818,1.888889,4.727273,50.850000,3.812778,1.075549
50%,0.500000,55.087500,4.080000,1.400000,22.625000,5.148352,2.269697,23.071795,2.333333,5.200000,54.984444,4.111806,1.400000
75%,0.642857,62.608333,4.454545,1.727273,25.750000,5.750000,2.848901,25.754808,2.727273,5.692308,59.302083,4.433333,1.714286
max,1.000000,84.316667,5.925000,3.000000,37.800000,8.333333,4.500000,34.600000,4.714286,8.250000,82.200000,5.466667,3.250000


#### Last-3-Weeks Variables:
Equal Weight Averages of last three observations.

In [44]:
# Extensive Summary Statistics

Last_3_Weeks_Variables = td.iloc[: , 32:45]
display(Last_3_Weeks_Variables.describe())

,hl3Win_PCT,hl3qbr,hl3rya,hl3to,hl3ps,hl33dc,hl3sa,hl3pa,hl3s,hl33dca,hl3qbra,hl3ryaa,hl3toa
count,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000,892.000000
mean,0.496048,54.620105,4.142975,1.404709,22.545217,5.140508,2.364350,23.024664,2.324365,5.208146,55.119544,4.169133,1.397235
std,0.212066,15.919499,0.809124,0.715400,6.523248,1.337792,1.099449,6.104742,1.018678,1.338709,14.241844,0.783037,0.728537
min,0.000000,10.366667,2.100000,0.000000,7.333333,1.333333,0.000000,4.000000,0.000000,1.000000,12.000000,1.966667,0.000000
25%,0.357143,43.958333,3.600000,1.000000,18.000000,4.333333,1.666667,18.666667,1.666667,4.333333,45.391667,3.633333,1.000000
50%,0.500000,54.516667,4.100000,1.333333,22.000000,5.000000,2.333333,22.666667,2.333333,5.000000,55.333333,4.133333,1.333333
75%,0.642857,66.016667,4.633333,2.000000,26.416667,6.000000,3.000000,27.333333,3.000000,6.000000,65.475000,4.666667,2.000000
max,1.000000,96.766667,6.900000,3.666667,48.000000,9.666667,6.666667,44.333333,6.000000,10.333333,97.333333,7.533333,3.666667


## Modeling

Across all different forecasting models, we will test the predicting capabilities of the 2 sets of distributed lag variables mentioned above, and will be mention in the python code as “_Model1” and “_Model2”. Model 1 incorporates the set of Year-to-Date variables that are an equal weight distributed average of all observations up to the present point in time; Model 2 incorporates the set of 3-week-average variables that are an equal weight average of the past 3 observations. 

For actual models, we will test 4 different models:

Standard OLS Model
Reduced Version of OLS
Probit Model
Logit Model

The first two models are straightforward. Both utilize basic OLS, with the second model being a more concise and reduced version of the first. From some preliminary analysis, it was shown that several variables may be statistically insignificant, so several of such variables which may also lack a strong theoretical significance are removed such that the adjusted R-squared values are slightly improved. The purpose of this is to test whether the list of variables overfit the model and cause the forecasts to not be generalizable onto new data. 

The third model is the Probit Model, which uses Maximum Likelihood estimation in order to estimate coefficients for the variables such that it minimizes residuals between the observed outcomes and the cumulative distribution function of the predicted y-hat values. In other terms, forecasted probabilities must be transformed using the normal distribution in the CDF, such that all predicted values fall between the range of 0 to 1.

Finally, the Logit Model uses a similar process as the Probit model, except predicted values are estimated using the logistic distribution. In this model, predicted values must be transformed into the difference of logs between P and 1-P, such that again the predicted values fall between 0-1. In simple terms, the model is as follows: log(P/(1-P)) = A + XB + e

For all four models, both sets of variables will be tested, which will pile up to a total of 8 distinct models. From these models, we will further analyze their efficiency and out-of-sample prediction accuracy. 


## Results & Findings

For the following sections the four models will be labeled as follow:

Standard Model: Standard OLS Model with full set of variables.

Concise Model: Reduced OLS Model with removed variables to check for overfitting.

Probit Model: Utilizes CDF from Normal Distribution to transform fitted values.

Logit Model: Utilizes CDF from Logistic Distribution to transform fitted values.

Furthermore for each form: Model 1 Represents the Year-to-date set of variables, Model 2 represents the Last-3-week set of variables.

### Standard Models

In [32]:
# Standard OLS Models (1. Using Year-to-Date Stats, 2. Using Last-3-week averages)

x1 = sm.add_constant(td.iloc[: , 6:32])
x2 = sm.add_constant(td.iloc[: , 32:58])
y1 = td['hwlBIN']
y2 = td['awlBIN']

mod_1 = sm.OLS(y1, x1)
mod_2 = sm.OLS(y1, x2)
res1 = mod_1.fit()
res2 = mod_2.fit()

# Calculating Model Adj. R-Squared, and RMSE (Results will be shown in Analysis Section)
standard_rsquared1 = res1.rsquared_adj
standard_rsquared2 = res2.rsquared_adj
standard_rmse1 = round((res1.ssr/892)**.5,4)
standard_rmse2 = round((res2.ssr/892)**.5,4)

print("Standard Model 1 R-Squared: " + str(round(standard_rsquared1,4)))
print("Standard Model 2 R-Squared: " + str(round(standard_rsquared2,4)))
print("Standard Model 1 RMSE: " + str(standard_rmse1))
print("Standard Model 2 RMSE: " + str(standard_rmse2))

Standard Model 1 R-Squared: 0.1501
Standard Model 2 R-Squared: 0.108
Standard Model 1 RMSE: 0.4496
Standard Model 2 RMSE: 0.4606


The results show that Model 1 (Year-to-date statistics) is superior to Model 2 (Last-3-weeks).

In [33]:
# Displaying Regression Results and Estimated Coefficients

model1_res = pd.concat([res1.params, res1.pvalues], axis=1)
model1_res.columns = ('Coefficient', 'P-value')
model2_res = pd.concat([res2.params, res2.pvalues], axis=1)
model2_res.columns = ('Coefficient', 'P-value')

print('Standard Model 1 Results:')
print('Model Adj. R-squared: ' + str(round(standard_rsquared1,4)))
print('\n')
display(model1_res)
print('\n')
print('Standard Model 2 Results:')
print('Model Adj. R-squared: ' + str(round(standard_rsquared2,4)))
print('\n')
display(model2_res)

Standard Model 1 Results:
Model Adj. R-squared: 0.1501




,Coefficient,P-value
const,1.813891,0.000450
hyWin_PCT,0.040148,0.798012
hyqbr,0.000755,0.777021
hyrya,-0.038805,0.251904
hyto,-0.108699,0.018453
hyps,0.012840,0.090562
hy3dc,-0.011625,0.608504
hysa,-0.043390,0.079092
hypa,-0.023450,0.002139
hys,0.014948,0.610295




Standard Model 2 Results:
Model Adj. R-squared: 0.108




,Coefficient,P-value
const,1.278738,0.000028
hl3Win_PCT,0.470199,0.000002
hl3qbr,-0.000101,0.949314
hl3rya,-0.021865,0.285934
hl3to,-0.009461,0.722157
hl3ps,0.001229,0.778086
hl33dc,-0.004010,0.774249
hl3sa,-0.032777,0.047014
hl3pa,-0.004487,0.297297
hl3s,-0.011394,0.517349


### Standard Model: Out-of-Sample Predictions (2019 & 2020 Test Data)

In [36]:
# Extracting Testing Period from Main Data Set
testdata = df[(df['Season']>2018) & (df['Week']>5) & (df['hwlBIN'] != 0.5)]

# Training Predictors and dependent var.
X1 = sm.add_constant(testdata.iloc[: , 6:32])
X2 = sm.add_constant(testdata.iloc[: , 32:58])
Y1 = testdata['hwlBIN']


# Predicted Values, and RMSE Calcumation
y_pred1 = res1.predict(X1)
SR1 = (Y1-y_pred1)**2
MSE1 = SR1.mean()
S_RMSE1 = round(MSE1**.5,4)

y_pred2 = res2.predict(X2)
SR2 = (Y1-y_pred2)**2
MSE2 = SR2.mean()
S_RMSE2 = round(MSE2**.5,4)

# RMSE results will be tabulated in Analysis Section below
# These are out-of-sample prediction RMSE
print("Standard Model 1 Out-of-Sample RMSE: " + str(S_RMSE1))
print("Standard Model 2 Out-of-Sample RMSE: " + str(S_RMSE2))


Standard Model 1 Out-of-Sample RMSE: 0.4638
Standard Model 2 Out-of-Sample RMSE: 0.4665


### Concise Model

In [37]:
# Dropping Several Insignificant Variables to test whether a concise model improves prediction due to Over-fitting

z1 = x1.drop(['hyWin_PCT', 'ayWin_PCT', 'hys', 'hysa', 'ays', 'aysa'], axis=1)
z2 = x2.drop(['hl3Win_PCT', 'al3Win_PCT', 'hl3s', 'hl3sa', 'al3s', 'al3sa'], axis=1)

# New OLS Regressions
smod_1 = sm.OLS(y1, sm.add_constant(z1))
smod_2 = sm.OLS(y1, sm.add_constant(z2))
sres1 = smod_1.fit()
sres2 = smod_2.fit()

# Calculating Model Adj. R-Squared, and RMSE (Results will be shown in Analysis Section)
concise_rsquared1 = round(sres1.rsquared_adj,4)
concise_rsquared2 = round(sres2.rsquared_adj,4)
concise_rmse1 = round((sres1.ssr/892)**.5,4)
concise_rmse2 = round((sres2.ssr/892)**.5,4)


smodel1_res = pd.concat([sres1.params, sres1.pvalues], axis=1)
smodel1_res.columns = ('Coefficient', 'P-value')
smodel2_res = pd.concat([sres2.params, sres2.pvalues], axis=1)
smodel2_res.columns = ('Coefficient', 'P-value')

print('Concise Model 1 Results:')
print('Model Adj. R-squared: ' + str(concise_rsquared1))
print('\n')
display(smodel1_res)
print('\n')
print('Standard Model 2 Results:')
print('Model Adj. R-squared: ' + str(concise_rsquared2))
print('\n')
display(smodel2_res)

Concise Model 1 Results:
Model Adj. R-squared: 0.1523




,Coefficient,P-value
const,1.822756,0.000010
hyqbr,0.001080,0.677654
hyrya,-0.044900,0.179356
hyto,-0.092545,0.034623
hyps,0.017105,0.006937
hy3dc,-0.014623,0.515074
hypa,-0.028294,0.000001
hy3dca,-0.037191,0.080741
hyqbra,0.011650,0.000067
hyryaa,-0.063004,0.068736




Standard Model 2 Results:
Model Adj. R-squared: 0.0699




,Coefficient,P-value
const,1.117585,0.000011
hl3qbr,0.000984,0.532700
hl3rya,-0.022098,0.288572
hl3to,-0.003151,0.906600
hl3ps,0.008971,0.024819
hl33dc,-0.009287,0.511867
hl3pa,-0.011865,0.003187
hl33dca,-0.018709,0.168088
hl3qbra,0.003743,0.017981
hl3ryaa,-0.035715,0.100603


### Concise Model: Out-of-Sample Prediction using 2019 & 2020 Data

In [39]:
# Training Predictors and dependent var.
Z1 = X1.drop(['hyWin_PCT', 'ayWin_PCT', 'hys', 'hysa', 'ays', 'aysa'], axis=1)
Z2 = X2.drop(['hl3Win_PCT', 'al3Win_PCT', 'hl3s', 'hl3sa', 'al3s', 'al3sa'], axis=1)

# Predicted Values, and RMSE Calcumation
y__pred1 = sres1.predict(Z1)
SR_1 = (Y1-y__pred1)**2
MSE_1 = SR_1.mean()
C_RMSE1 = round(MSE_1**.5,4)

y__pred2 = sres2.predict(Z2)
SR_2 = (Y1-y__pred2)**2
MSE_2 = SR_2.mean()
C_RMSE2 = round(MSE_2**.5,4)


# RMSE results will be tabulated in Analysis Section below
print("Concise Model 1 Out-of-sample RMSE: " + str(C_RMSE1))
print("Concise Model 2 Out-of-sample RMSE: " + str(C_RMSE2))


Concise Model 1 Out-of-sample RMSE: 0.463
Concise Model 2 Out-of-sample RMSE: 0.4761


### Probit Model

In [40]:
# Probit Regression Model With Full Set of Regressors

pmod_1 = sm.Probit(y1, x1)
pmod_2 = sm.Probit(y1, x2)
pres1 = pmod_1.fit()
pres2 = pmod_2.fit()

pmodel1_res = pd.concat([pres1.params, pres1.pvalues], axis=1)
pmodel1_res.columns = ('Coefficient', 'P-value')
pmodel2_res = pd.concat([pres2.params, pres2.pvalues], axis=1)
pmodel2_res.columns = ('Coefficient', 'P-value')
print('\n')
print('\n')

print('Probit Model 1 Results:')
print('\n')
display(pmodel1_res)
print('\n')
print('Probit Model 2 Results:')
print('\n')
display(pmodel2_res)

Optimization terminated successfully.
         Current function value: 0.587741
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.611925
         Iterations 5




Probit Model 1 Results:




,Coefficient,P-value
const,3.954770,0.009417
hyWin_PCT,0.078785,0.863793
hyqbr,0.002736,0.727426
hyrya,-0.122370,0.220217
hyto,-0.324200,0.016523
hyps,0.037370,0.092570
hy3dc,-0.031263,0.638251
hysa,-0.130635,0.073165
hypa,-0.069480,0.001989
hys,0.038178,0.658006




Probit Model 2 Results:




,Coefficient,P-value
const,2.281801,0.008308
hl3Win_PCT,1.299936,0.000003
hl3qbr,-0.000255,0.954491
hl3rya,-0.068953,0.235527
hl3to,-0.024584,0.742900
hl3ps,0.004910,0.692713
hl33dc,-0.011770,0.763347
hl3sa,-0.090023,0.052154
hl3pa,-0.012987,0.285752
hl3s,-0.033034,0.507250


Generating Fitted Values, R-Squared, and RMSE

In [41]:
# Fitted Values
p_yh1=pres1.fittedvalues
p_yh2=pres2.fittedvalues

# As we can see, Predicted values exceed (0,1), which should not be the case for a binary dependent variable
# The Probit model attempts to fix this by normalizing predicted values
print(p_yh1.max())
print(p_yh1.min())

1.7267583149203678
-1.831139505699401


In [42]:
# Predicted Values are Normalized using the Cumulative Distribution Function of the Normal

normalized_predictions_1 = norm.cdf(p_yh1)
normalized_predictions_2 = norm.cdf(p_yh2)


# Results/Probabilities no longer exceed (0,1)
print(normalized_predictions_1.max())
print(normalized_predictions_1.min())
print(normalized_predictions_2.max())
print(normalized_predictions_2.min())

0.9578944611208641
0.03353986162421947
0.9474629292079905
0.06909636409581645


Calculating RMSE and Pseudo R-Squared

In [43]:
# Finding RMSE

probit_SR1 = (y1 - normalized_predictions_1)**2
probit_SR2 = (y1 - normalized_predictions_2)**2

probit_SSR1 = probit_SR1.sum()
probit_SSR2 = probit_SR2.sum()

Probit_RMSE1 = round((probit_SSR1/892)**.5,4)
Probit_RMSE2 = round((probit_SSR2/892)**.5,4)

print("Probit Model 1 RMSE: " + str(Probit_RMSE1))
print("Probit Model 2 RMSE: " + str(Probit_RMSE2))


Probit Model 1 RMSE: 0.449
Probit Model 2 RMSE: 0.4602


In [45]:
# Finding R-Squared

ST = (y1 - y1.mean())**2

SST = ST.sum()

probit_Rsq1 = round((SST-probit_SSR1)/SST,4)

probit_Rsq2 = round((SST-probit_SSR2)/SST,4)

print("Probit Model 1 R-Squared: " + str(probit_Rsq1))
print("Probit Model 2 R-Squared: " + str(probit_Rsq2))

Probit Model 1 R-Squared: 0.1773
Probit Model 2 R-Squared: 0.1355


### Probit Model: Out-of-Sample Prediction using 2019 & 2020 Data

In [46]:
# Predicting using 2020 Observed Values of regressors

probit_y_pred1 = pres1.predict(X1)
pSR1 = (Y1-probit_y_pred1)**2
pMSE1 = pSR1.mean()
P_RMSE1 = round(pMSE1**.5,4)

probit_y_pred2 = pres2.predict(X2)
pSR2 = (Y1-probit_y_pred2)**2
pMSE2 = pSR2.mean()
P_RMSE2 = round(pMSE2**.5,4)


# RMSE results will be tabulated in Analysis Section below

print("Probit Model 1 Out-of-sample RMSE: " + str(P_RMSE1))
print("Probit Model 2 Out-of-sample RMSE: " + str(P_RMSE2))


Probit Model 1 Out-of-sample RMSE: 0.4624
Probit Model 2 Out-of-sample RMSE: 0.465


### LOGIT MODEL

In [47]:
# Logit Regression Model With Full Set of Regressors

lmod_1 = sm.Logit(y1, x1)
lmod_2 = sm.Logit(y1, x2)
lres1 = lmod_1.fit()
lres2 = lmod_2.fit()

lmodel1_res = pd.concat([lres1.params, lres1.pvalues], axis=1)
lmodel1_res.columns = ('Coefficient', 'P-value')
lmodel2_res = pd.concat([lres2.params, lres2.pvalues], axis=1)
lmodel2_res.columns = ('Coefficient', 'P-value')

print('\n')
print('\n')

print('Logit Model 1 Results:')
print('\n')
display(lmodel1_res)
print('\n')
print('Logit Model 2 Results:')
print('\n')
display(lmodel2_res)

Optimization terminated successfully.
         Current function value: 0.587722
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.611827
         Iterations 5




Logit Model 1 Results:




,Coefficient,P-value
const,6.590972,0.009118
hyWin_PCT,0.174847,0.820016
hyqbr,0.003550,0.788071
hyrya,-0.204265,0.219500
hyto,-0.527630,0.019003
hyps,0.065105,0.081025
hy3dc,-0.053209,0.633313
hysa,-0.213447,0.079277
hypa,-0.116862,0.001853
hys,0.066907,0.645154




Logit Model 2 Results:




,Coefficient,P-value
const,3.692729,0.009510
hl3Win_PCT,2.180497,0.000003
hl3qbr,-0.000452,0.950940
hl3rya,-0.111952,0.241968
hl3to,-0.041203,0.738928
hl3ps,0.007334,0.718776
hl33dc,-0.019778,0.760318
hl3sa,-0.147823,0.052679
hl3pa,-0.021495,0.285147
hl3s,-0.056046,0.493301


Generate Fitted Values, R-Squared, and RMSE

In [48]:
# Finding Fitted Values and Model RMSE

l_yh1 = lres1.fittedvalues
l_yh2 = lres2.fittedvalues

logit_prob_1 = e**(l_yh1)/(1+e**(l_yh1))
logit_prob_2 = e**(l_yh2)/(1+e**(l_yh2))

logit_SR1 = (y1 - logit_prob_1)**2
logit_SR2 = (y1 - logit_prob_2)**2

logit_SSR1 = logit_SR1.sum()
logit_SSR2 = logit_SR2.sum()

logit_RMSE1 = round((logit_SSR1/892)**.5,4)
logit_RMSE2 = round((logit_SSR2/892)**.5,4)

print("Logit Model 1 RMSE: " + str(logit_RMSE1))
print("Logit Model 2 RMSE: " + str(logit_RMSE2))

Logit Model 1 RMSE: 0.4488
Logit Model 2 RMSE: 0.4601


In [50]:
# Finding Model R-squared

logit_Rsq1 = round((SST-logit_SSR1)/SST,4)

logit_Rsq2 = round((SST-logit_SSR2)/SST,4)

print("Logit Model 1 R-Squared: " + str(logit_Rsq1))
print("Logit Model 2 R-Squared: " + str(logit_Rsq2))

Logit Model 1 R-Squared: 0.1778
Logit Model 2 R-Squared: 0.1358


#### Out-of-Sample Logit Predictions for 2019 & 2020

In [52]:
# Predictions for 2019 & 2020

logit_y_pred1 = lres1.predict(X1)
lSR1 = (Y1-logit_y_pred1)**2
lMSE1 = lSR1.mean()
L_RMSE1 = round(lMSE1**.5,4)

logit_y_pred2 = lres2.predict(X2)
lSR2 = (Y1-logit_y_pred2)**2
lMSE2 = lSR2.mean()
L_RMSE2 = round(lMSE2**.5,4)

# Out of sample Prediction RMSE

print("Logit Model 1 Out-of-sample RMSE: " + str(L_RMSE1))
print("Logit Model 2 Out-of-sample RMSE: " + str(L_RMSE2))

Logit Model 1 Out-of-sample RMSE: 0.4623
Logit Model 2 Out-of-sample RMSE: 0.4648


## Model Analysis and RMSE Comparison

In [53]:
data = [[standard_rmse1, standard_rmse2, concise_rmse1, concise_rmse2, Probit_RMSE1, Probit_RMSE2, logit_RMSE1, logit_RMSE2], 
       [S_RMSE1, S_RMSE2, C_RMSE1, C_RMSE2, P_RMSE1, P_RMSE2, L_RMSE1, L_RMSE2]]

df = pd.DataFrame(data, columns = ['Standard Model 1', 'Standard Model 2', 'Concise Model 1', 'Concise Model 2', 'Probit Model 1', 'Probit Model 2', 'Logistic Model 1', 'Logistic Model 2'])
df.index = ['Model RMSE', 'Out-of-Sample Prediction RMSE']


display(df)




,Standard Model 1,Standard Model 2,Concise Model 1,Concise Model 2,Probit Model 1,Probit Model 2,Logistic Model 1,Logistic Model 2
Model RMSE,0.4496,0.4606,0.4506,0.4720,0.4490,0.4602,0.4488,0.4601
Out-of-Sample Prediction RMSE,0.4638,0.4665,0.4630,0.4761,0.4624,0.4650,0.4623,0.4648


The RMSE results of all estimated models are included in the table above. It is evident that across all cases, Model 1 outperforms Model 2 in both model accuracy and out-of-sample prediction accuracy (by having a lower RMSE). This finding is in-line with preliminary hypotheses -- that Year-to-date stats would be a better regressor that Last-3-weeks stats.
    
(An additional note: All RMSE of the 16 models are comparable because they share the same dependent variable. Even though Probit and Logit transform fitted and predicted values, the actual observed y is unchanged).
    


In [54]:
data2 = [[standard_rmse1, concise_rmse1, Probit_RMSE1, logit_RMSE1], 
       [S_RMSE1, C_RMSE1, P_RMSE1, L_RMSE1]]

df2 = pd.DataFrame(data2, columns = ['Standard Model 1', 'Concise Model 1', 'Probit Model 1','Logistic Model 1'])
df2.index = ['Model RMSE', 'Out-of-Sample Prediction RMSE']

print("Table with only Model 1 Results")
display(df2)


Table with only Model 1 Results


,Standard Model 1,Concise Model 1,Probit Model 1,Logistic Model 1
Model RMSE,0.4496,0.4506,0.4490,0.4488
Out-of-Sample Prediction RMSE,0.4638,0.4630,0.4624,0.4623


Next, we compare between different types of Models, focusing on Model 1 -- we find, although by a small margin, utilizing Probit and Logit models does produce better results in-sample and out-of-sample (with Logit Model producing the best results). This is to be expected as it limits prediction to within the 0-1 range, which is far more appropriate for a binary dependent variable, and predictions which represent the probability of events occuring.
    
Between the basic OLS models themselves, the results show that removing several insignificant variables do not lead to better RMSE, even though the Adj. R-squared is slightly improved. The fact that out-of-sample prediction does not improve shows that overfitting is not an issue that plagues this model, and therefore we keep the full set of data for the Probit and Logit model estimations.

### Comparing Optimal Logit Model to Betting Market Data

From our results, it is determined that the logit model utilizing the year-to-date set of variables produce the lowest error in out-of-sample prediction, and thus we will utlize that model to compare with actual sportsbook and betting market odds to detect whether this logit model is capable of producing results that outperform betting houses.

In [55]:
odds = td['hodds']

sportsbook_model = sm.Logit(y1, sm.add_constant(odds))
sb_res = sportsbook_model.fit()

sb_res_table = pd.concat([sb_res.params, sb_res.pvalues], axis=1)
sb_res_table.columns = ('Coefficient', 'P-value')

print("Logit Model using Odds as regressor")

display(sb_res_table)

# Note Odds is theoretically inversely proportional to probability of winning. 
# Worse teams have higher odds (or higher payout if win).

Optimization terminated successfully.
         Current function value: 0.613835
         Iterations 6
Logit Model using Odds as regressor


,Coefficient,P-value
const,2.144538,6.818319e-26
hodds,-0.950537,3.939747e-21


In [56]:
# Calculating Model RMSE for the logit model using sportsbook odds as explanatory variable

sb_yh = sb_res.fittedvalues

sb_prob = e**(sb_yh)/(1+e**(sb_yh))

sb_SR1 = (y1 - sb_prob)**2

sb_SSR1 = sb_SR1.sum()

logit_sb_RMSE1 = round((sb_SSR1/892)**.5,4)

print("Logit Model w/ Sportsbook Data RMSE: " + str(logit_sb_RMSE1))

Logit Model w/ Sportsbook Data RMSE: 0.4595


Out-of-sample Logit Predictions with Sportsbook Data

In [58]:
# Predictions for 2019 & 2020


f_odds = testdata['hodds']
sb_y_pred1 = sb_res.predict(sm.add_constant(f_odds))
sbSR1 = (Y1-sb_y_pred1)**2
sbMSE1 = sbSR1.mean()
sb_RMSE1 = round(sbMSE1**.5,4)


# Out of sample Prediction RMSE
print("Logit Model w/ Sportsbook Data Out-of-sample RMSE: " + str(sb_RMSE1))


Logit Model w/ Sportsbook Data Out-of-sample RMSE: 0.4537


Comparing Logit Models between Year-to-date statistics and betting odds

In [59]:
comp = [[logit_RMSE1,logit_sb_RMSE1],[L_RMSE1,sb_RMSE1]]

comp = pd.DataFrame(comp, columns = ['Logit Model 1', 'Logit Model Sportsbook'])
comp.index = ['Model RMSE', 'Out-of-Sample Prediction RMSE']

display(comp)

,Logit Model 1,Logit Model Sportsbook
Model RMSE,0.4488,0.4595
Out-of-Sample Prediction RMSE,0.4623,0.4537


The main takeaway from this table is the fact that although the logistic model utilizing a set of explanatory statistics has a lower Model RMSE, it fails to predict out-of-sample results more accurately than pure betting market odds. 

However, this does not mean that the model is completely useless. It is possible that errors are exacerbated as a result of a few outliers, which negatively effect RMSE as a whole, but could still mean a good model for majority of predictions.

We test this theory by simulating actual bets made with our prediction model:

#### Exploring Possible Strategy Payout

With any odds posted by betting companies, an associated probability comes with that odds value such that they are direct inverses of one another. Higher Probability events have lower odds (or payout) and vice versa.

Ex. 33.3% probability events have an odds value of 3.0.

Thus we compute arbitrage values -- the difference between implied probability from betting odds and predicted probability from logit model

We summarize simulated betting profit on instances where we observe a 10%, 15%, 20%, 25%, & 30% arbitrage value (Sindre Hansen refers to this as the "Value Threshold").

In [60]:
res = testdata['hwlBIN']
sb_prob = testdata['hprob']

strat_df = pd.concat([res, f_odds, sb_prob, logit_y_pred1], axis = 1)
strat_df.columns = ('Result', 'Odds', 'Sportsbook', 'Prediction')

# Identifying Discrepancies in predicted probabilities vs. Gambling Odds
strat_df['Arbitrage'] = logit_y_pred1 - sb_prob

# Simulating Betting profit from a fixed bet on each game
strat_df['Bet Profit'] = (f_odds-1)*res + (res-1)

v10 = strat_df[strat_df['Arbitrage']>0.10]
v15 = strat_df[strat_df['Arbitrage']>0.15]
v20 = strat_df[strat_df['Arbitrage']>0.20]
v25 = strat_df[strat_df['Arbitrage']>0.25]
v30 = strat_df[strat_df['Arbitrage']>0.30]

print("Sample Table of Simulated Bets for min. 10% Arbitrage Value")
display(v10)

Sample Table of Simulated Bets for min. 10% Arbitrage Value


,Result,Odds,Sportsbook,Prediction,Arbitrage,Bet Profit
1359,0.0,2.100000,0.476190,0.578622,0.102431,-1.000000
1367,1.0,2.400000,0.416667,0.564282,0.147615,1.400000
1368,1.0,1.869565,0.534884,0.664739,0.129855,0.869565
1373,0.0,2.450000,0.408163,0.658888,0.250724,-1.000000
1380,0.0,4.750000,0.210526,0.465344,0.254818,-1.000000
...,...,...,...,...,...,...
1779,0.0,2.500000,0.400000,0.502537,0.102537,-1.000000
1780,1.0,1.588235,0.629630,0.773143,0.143514,0.588235
1786,0.0,3.500000,0.285714,0.787365,0.501651,-1.000000
1787,1.0,1.800000,0.555556,0.712239,0.156684,0.800000


In [61]:
# Creating Summary Table of simulated bets at all studied Arbitrage Values (Thresholds)
bet_data = [[v10['Bet Profit'].mean(),v15['Bet Profit'].mean(),v20['Bet Profit'].mean(),v25['Bet Profit'].mean(),v30['Bet Profit'].mean()],
            [v10['Bet Profit'].count(),v15['Bet Profit'].count(),v20['Bet Profit'].count(),v25['Bet Profit'].count(),v30['Bet Profit'].count()]]
bet_df = pd.DataFrame(bet_data, columns = ['v10', 'v15', 'v20', 'v25', 'v30',])
bet_df.index = ['Average Return', 'Count']


print("Average Return on Bets Made")
display(bet_df)

Average Return on Bets Made


,v10,v15,v20,v25,v30
Average Return,-0.045172,-0.036821,0.040494,0.121251,0.127778
Count,98.000000,61.000000,44.000000,22.000000,9.000000


As we can see, lower thresholds do not create positive returns, however, contrary to what Hansen finds in his paper on the Premier League study for soccer, in this research, we find that higher arbitrage thresholds do provide positive returns on bets made -- with the expected return increasing with respect to the size of the threshold. This is consistent with logic: if our logistic model predicts a much higher probability of a win then the odds reflect, those bets should be relatively profitable.

Similar to the literature however, we note the lack of large sample size in the higher thresholds, and we reserve judgement on the accuracy of simulated bets due to the possibility of luck. As a side effect of a large arbitrage value, it is likely that for those games, the odds are extremely low from the sportsbook, which means that they have a very good payout on bets made. As a result, with sufficient luck, average returns can swing in the positive direction very quickly with only a few samples to examine.

But at the 20% arbitrage level, we do find 44 observations, and around a 4.05% return on bets made. 

These results are further encouraging for future research because it suggests that even if the model cannot outperform sportsbook odds in minimizing prediction error, by utlizing an optimal arbitrage value/threshold and strategy can still lead to profitable results.

# Conclusion

From our results, we make several key conclusions regarding the process of identifying the optimal model to forecast NFL game results, as well as identify several weaknesses of this methodology as well as areas for future research and model optimization.

First, we find that using a longer distributed lag (or equal weight averages) for our set of variables produces better results both in terms of model RMSE, and Out-of-sample prediction accuracy. The simple interpretation of this finding is that long-run statistics are a better indicator of a team’s current state rather than short-run recent statistics, which doesn’t necessarily match conventional thinking of when a team may be “hot” or “on a streak”. This area remains open to further research, however, both in the possibility of including a longer period of weeks for the short-run set of variables, but also a possible imbalance weighted average, where more recent performances may carry a larger weight.

Second, we find that the logit model is the best performer amongst the comparisons of the Probit and OLS models. This follows our general hypothesis, since Logit Models account for the binary or limited nature of the dependent variable by constraining predicted values, and general literature indicates that the logit model performs the best in prediction capabilities.

Finally, although we find that our model with sets of variables fails to outperform a simple one variable logit model using actual betting odds, we find that profitable strategies for identifying arbitrage opportunities may still be available. We test simulated bets for games at separate arbitrage values or thresholds (difference between predicted probability and implied probability of odds), and find that at the higher values/thresholds, the strategy is able to achieve positive returns, although with low sample size.

An additional study for future research would be to continually test this model upon new seasons of testing data, for example 2021 & 2022 data. Furthermore, with more years of data available, it is also possible to adjust the size of not only our testing set, but the training data as well. For our project, we utilize all data available collected from 2014 - 2018 for the model, but it is possible that using training data more recent to the testing data may yield more accurate coefficients.

For example, to predict 2020 results, it may be more accurate to use 2016-2019 as the training set because it may be a better indicator of the nature of the game in 2020. This is due to several factors, but the main one is the ever-evolving state, rules, and player demographic of the game. As the nature of the game shifts ever slightly each year, it is plausible to expect the coefficients to be non-constant across a long period of time. Thus, by using more recent training data, model fit on out-of-sample predictions may be improved.

Finally, for an obvious point of improvement of the model, a more in-depth selection process of variables can be implemented to further optimize the basic logit model. For our project, we selected several variables that were theoretically important in game outcomes as well as several that were statistically significant in preliminary analysis. The basic logit model had an R-squared value of around 15%, which suggests a large area of improvement even if some variance in outcome may be random and unpredictable. Even at the current level of model efficiency, it is proven that positive return strategies can be developed in regards to the betting market, and if the base logit model is improved, results on predictions can be drastically improved.


## References


Hansen, Sindre. “Econometrics versus the Bookmakers: An Econometric Approach to Sports Betting.” The Arctic University of Norway, 2016. 

“NFL Football Scores - NFL Scoreboard.” [Database]. ESPN, ESPN Internet Ventures, https://www.espn.com/nfl/scoreboard. 

“Sportsbook Reviews.” [Database]. Historical NFL Scores and Odds Archives, https://www.sportsbookreviewsonline.com/scoresoddsarchives/nfl/nfloddsarchives.htm. 

Torres-Reyna, Oscar. “Logit, Probit and Multinomial Logit Models in R.” Dec. 2014, Princeton University. 

Wilkens, Sascha. "Sports Prediction and Betting Models in the Machine Learning Age: The Case of Tennis." 1 Jan. 2021 : 99 – 117.
